In [ ]:
import pandas as pd
from scipy.stats import qmc
import random
import time
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init
import itertools

In [ ]:
qs = {
    "methanol" : 1.4320,
    "ethanol" : 2.5880,
    "2-propanol" : 3.1240,
    "acetone" : 2.3360,
    "mibk" : 3.9520,
    "ethyl-acetate" : 3.1160,
    "chloroform" : 2.41,
    "toluene" : 2.9680,
    "water" : 1.4,
    "n-pentane" : 3.316,
    "n-heptane" : 4.396,
    "1-propanol" : 3.128,
    "1-pentanol" : 4.208,
    "1-butanol" : 3.668,
}

rs = {
    "methanol" : 1.4311,
    "ethanol" : 2.5755,
    "2-propanol" : 3.2491,
    "acetone" : 2.5735,
    "mibk" : 4.5959,
    "ethyl-acetate" : 3.4786,
    "chloroform" : 2.87,
    "toluene" : 3.9228,
    "water" : 0.92,
    "n-pentane" : 3.8254,
    "n-heptane" : 5.1742,
    "1-propanol" : 3.2499,
    "1-pentanol" : 4.5987,
    "1-butanol" : 3.9243,
}

In [ ]:
inv_qs = dict(map(reversed, qs.items()))

In [ ]:
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,
                  GC={'H2O':1}) # water

npentane = component(name='n-pentane', Tc=469.7, Pc=33.7, Zc=0.27, Vc=313,
                     GC={'CH3':2, 'CH2':3}) # n-pentane

nheptane = component(name='n-heptane', Tc=540.2, Pc=27.4, Zc=0.261, Vc=428,
                     GC={'CH3':2, 'CH2':5}) # n-heptane

ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168,
                    GC={'CH3':1, 'CH2':1, 'OH':1}) # ethanol

_1propanol = component(name='1-propanol', Tc=536.8, Pc=51.69, Zc=0.254, Vc=219,
                       GC={'CH3':1, 'CH2':2, 'OH':1}) # 1-propanol

_1butanol = component(name='1-butanol', Tc=563.1, Pc=44.14, Zc=0.258, Vc=273,
                      GC={'CH3':1, 'CH2':3, 'OH':1})

_1pentanol = component(name='1-pentanol', Tc=588.1, Pc=38.97, Zc=0.258, Vc=326,
                       GC={'CH3':1, 'CH2':4, 'OH':1})

acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213,
                    GC={'CH3CO':1, 'CH3':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1})

In [ ]:
component_list = [water,npentane,nheptane,ethanol,_1propanol,_1butanol,_1pentanol,acetone]

In [ ]:
binary_mix = itertools.permutations(component_list, 2) #ordering does not matter #outputs every combination of mixtures give a list

In [ ]:
binary_mix= list(binary_mix)
binary_mix_no = len(binary_mix) #number of different mixtures
print('binary combinations', binary_mix_no)

n=0
binary_mix_list = [] #list of binary mixtures

while n <= (binary_mix_no-1):
    mix = mixture(binary_mix[n][0],binary_mix[n][1])
    #print('mixture index:',n, 'Component names:', mix.names)
    binary_mix_list.append(mix)
    n=n+1

In [ ]:
num_samples_base2 = 12

In [ ]:
st = time.time() #start timer
Tmin = 293.15
Tmax = 318.15
sampler_sob = qmc.Sobol(d=4, scramble=True)
sample_sob = sampler_sob.random_base2(num_samples_base2)
final_sample_list = []
for sample in sample_sob:
    x1 = sample[0]/(sample[0] + sample[1] + sample[2])
    x2 = sample[1]/(sample[0] + sample[1] + sample[2])
    x3 = sample[2]/(sample[0] + sample[1] + sample[2])
    Tsob = (sample[3] * (Tmax-Tmin) + Tmin)
    final_sample_list.append([x1, x2, Tsob])
et = time.time()
sobol_time = et - st
print('Time taken for Sobol sequence generation: ', sobol_time)

In [ ]:
st = time.time()
master_list = []
for i in binary_mix:
    s1 = i[0].name
    s2 = i[1].name
    q1 = qs[s1]
    q2 = qs[s2]
    r1 = rs[s1]
    r2 = rs[s2]
    for j in final_sample_list:
        master_list.append([s1,s2,q1,q2,r1,r2,j[0],j[1],j[2]])
et = time.time()
list_time = et - st
print('Time taken for list generation: ', list_time)

In [ ]:
master_list_column_names = ['Solvent 1', 'Solvent 2', 'q1', 'q2', 'r1', 'r2', 'x1', 'x2', 'T']
df = pd.DataFrame(master_list, columns=master_list_column_names)
df.to_csv('Case2.csv')
print('Total time for generating data: ', sobol_time + list_time)